In [8]:
import pandas as pd
from datetime import datetime

In [9]:
def initialize_budget():
    while True:
        try:
            budget = float(input("Enter your total budget: "))
            if budget > 0:
                return budget
            print("Budget must be positive!")
        except ValueError:
            print("Please enter a valid number.")

In [10]:
def expenses(budget, total_budget, category, amount):
    category = category.lower()
    if amount <= 0:
        print("Amount must be positive!")
        return budget
    if category not in categories:
        print(f"Invalid category! Choose from {categories}")
        return budget
    
    if amount > budget:
        print("Transaction failed: You have exceeded your budget!")
        print("Options:")
        print("1. Add to your budget")
        print("2. Take a loan from eSewa")
        choice = input("Enter choice (1 or 2): ").strip()
        
        if choice == "1":
            while True:
                try:
                    additional_budget = float(input("Enter amount to add to budget: "))
                    if additional_budget > 0:
                        break
                    print("Amount must be positive!")
                except ValueError:
                    print("Please enter a valid number.")
            budget += additional_budget
            total_budget += additional_budget
            transactions.append({
                "date": datetime.now().strftime("%Y-%m-%d"),
                "category": "budget_increase",
                "amount": additional_budget,
                "total_budget": total_budget,
                "remaining_balance": budget
            })
            print(f"Budget increased by {additional_budget}. New total budget: {total_budget}")
            # Retry the expense with updated budget
            budget = expenses(budget, total_budget, category, amount)
            return budget
        
        elif choice == "2":
            while True:
                try:
                    loan_amount = float(input("Enter loan amount from eSewa: "))
                    if loan_amount > 0:
                        break
                    print("Loan amount must be positive!")
                except ValueError:
                    print("Please enter a valid number.")
            budget += loan_amount
            total_budget += loan_amount
            # Calculate interest (5% per year)
            interest_rate = 0.05
            total_loan_with_interest = loan_amount * (1 + interest_rate)
            monthly_payment = total_loan_with_interest / 12
            transactions.append({
                "date": datetime.now().strftime("%Y-%m-%d"),
                "category": "esewa_loan",
                "amount": loan_amount,
                "total_budget": total_budget,
                "remaining_balance": budget
            })
            print(f"Loan of {loan_amount} taken from eSewa.")
            print(f"You must pay {total_loan_with_interest:.2f} (including 5% interest) in one year.")
            print(f"Monthly payment for 12 months: {monthly_payment:.2f}")
            # Retry the expense with updated budget
            budget = expenses(budget, total_budget, category, amount)
            return budget
        
        else:
            print("Invalid choice! Transaction cancelled.")
            return budget
    
    budget -= amount
    transactions.append({
        "date": datetime.now().strftime("%Y-%m-%d"),
        "category": category,
        "amount": amount,
        "total_budget": total_budget,
        "remaining_balance": budget
    })
    print(f"Expense added: {amount} in {category}. Remaining balance: {budget}")
    return budget


In [11]:
def get_transactions(total_budget, remaining_balance):
    if not transactions:
        print("No transactions.")
        return
    
    df = pd.DataFrame(transactions)
    category_totals = df.groupby('category')['amount'].sum().reset_index()
    category_totals.columns = ['category', 'total_spent']
    
    with open("transaction.csv", "w") as f:
        f.write(f"Total Budget,{total_budget}\n\n")
        f.write("Transactions\n")
        df.to_csv(f, index=False, header=True)
        f.write("\nCategory Totals\n")
        if not category_totals.empty:
            category_totals.to_csv(f, index=False, header=True)
        else:
            f.write("No transactions.\n")
        f.write(f"\nRemaining Balance,{remaining_balance}\n")
    
    print("Saved to transaction.csv")

In [12]:
def main():
    total_budget = initialize_budget()
    budget = total_budget
    
    while True:
        print("\n1. Add new category")
        print("2. Select existing category")
        print("3. Save and exit")
        choice = input("Enter choice (1, 2, or 3): ")
        
        if choice == "1":
            category = input("Enter new category: ").strip().lower()
            if not category:
                print("Category cannot be empty!")
            elif category in categories:
                print("Category already exists!")
            else:
                categories.add(category)
                print(f"Added category: {category}")
                while True:
                    try:
                        amount = float(input("Enter amount for this category: ").strip())
                        break
                    except ValueError:
                        print("Please enter a valid amount (integer or float).")
                budget = expenses(budget, total_budget, category, amount)
            
        elif choice == "2":
            if not categories:
                print("No categories available! Please add a category first.")
                continue
            category = input(f"Enter category {categories}: ").lower()
            while True:
                try:
                    amount = float(input("Enter amount: ").strip())
                    break
                except ValueError:
                    print("Please enter a valid amount (integer or float).")
            budget = expenses(budget, total_budget, category, amount)
            
        elif choice == "3":
            get_transactions(total_budget, budget)
            print(f"Total budget: {total_budget}")
            print(f"Final balance: {budget}")
            break
            
        else:
            print("Please enter 1, 2, or 3.")

categories = set()
transactions = []
main()

Enter your total budget: 12000

1. Add new category
2. Select existing category
3. Save and exit
Enter choice (1, 2, or 3): 1
Enter new category: food
Added category: food
Enter amount for this category: 4500
Expense added: 4500.0 in food. Remaining balance: 7500.0

1. Add new category
2. Select existing category
3. Save and exit
Enter choice (1, 2, or 3): 1
Enter new category: bills
Added category: bills
Enter amount for this category: 7000
Expense added: 7000.0 in bills. Remaining balance: 500.0

1. Add new category
2. Select existing category
3. Save and exit
Enter choice (1, 2, or 3): 1
Enter new category: travel
Added category: travel
Enter amount for this category: 3000
Transaction failed: You have exceeded your budget!
Options:
1. Add to your budget
2. Take a loan from eSewa
Enter choice (1 or 2): 1
Enter amount to add to budget: 5000
Budget increased by 5000.0. New total budget: 17000.0
Expense added: 3000.0 in travel. Remaining balance: 2500.0

1. Add new category
2. Select ex